# Employee Salary Dataset Generation

This notebook generates a comprehensive, realistic employee salary dataset for the Employee Salary Prediction System. The dataset includes various features that influence salary determination in the Indian job market.

## Objectives:
1. **Create realistic employee data** with proper distributions and correlations
2. **Generate salary values in INR** based on multiple factors
3. **Include data inconsistencies** to simulate real-world scenarios
4. **Apply business logic** for salary calculation
5. **Save the dataset** for further analysis and modeling

## Dataset Features:
- **Personal**: Age, Gender, Education Level
- **Professional**: Years of Experience, Job Title, Department, Seniority Level
- **Geographic**: City, State, Region
- **Skills**: Technical Skills, Certifications, Language Skills
- **Company**: Company Size, Industry, Company Type
- **Performance**: Performance Rating, Bonus Percentage
- **Target**: Annual Salary (INR)

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
from datetime import datetime, timedelta
import warnings
import os
import sys

# Add src directory to path for imports
sys.path.append('../src')

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🌊 Seaborn version: {sns.__version__}")

In [ ]:
# Configuration and Parameters
CONFIG = {
    'dataset_size': 10000,  # Number of employees to generate
    'missing_rate': 0.05,   # Percentage of missing values to introduce
    'outlier_rate': 0.02,   # Percentage of outliers to introduce
    'random_state': 42,
    'save_path': '../data/raw/',
    'currency': 'INR'
}

# Define categorical variables and their distributions
GENDER_DIST = ['Male', 'Female', 'Other']
GENDER_WEIGHTS = [0.65, 0.34, 0.01]

EDUCATION_LEVELS = [
    'High School', 'Diploma', 'Bachelor', 'Master', 'PhD'
]
EDUCATION_WEIGHTS = [0.05, 0.15, 0.50, 0.25, 0.05]

JOB_TITLES = {
    'Technology': [
        'Software Engineer', 'Senior Software Engineer', 'Tech Lead', 'Engineering Manager',
        'Data Scientist', 'ML Engineer', 'DevOps Engineer', 'Product Manager',
        'UI/UX Designer', 'Quality Analyst', 'System Administrator', 'Database Administrator'
    ],
    'Finance': [
        'Financial Analyst', 'Senior Financial Analyst', 'Finance Manager', 'CFO',
        'Accountant', 'Senior Accountant', 'Investment Analyst', 'Risk Analyst'
    ],
    'Marketing': [
        'Marketing Specialist', 'Marketing Manager', 'Digital Marketing Manager',
        'Brand Manager', 'Content Creator', 'SEO Specialist', 'Social Media Manager'
    ],
    'Sales': [
        'Sales Representative', 'Senior Sales Representative', 'Sales Manager',
        'Business Development Manager', 'Account Manager', 'Sales Director'
    ],
    'HR': [
        'HR Specialist', 'HR Manager', 'Talent Acquisition Specialist',
        'HR Business Partner', 'Compensation Analyst', 'Training Manager'
    ],
    'Operations': [
        'Operations Analyst', 'Operations Manager', 'Supply Chain Manager',
        'Project Manager', 'Business Analyst', 'Process Improvement Specialist'
    ]
}

INDIAN_CITIES = {
    'Tier 1': ['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Chennai', 'Pune', 'Kolkata'],
    'Tier 2': ['Ahmedabad', 'Jaipur', 'Surat', 'Lucknow', 'Kanpur', 'Nagpur', 'Indore', 'Coimbatore'],
    'Tier 3': ['Vadodara', 'Bhopal', 'Ludhiana', 'Agra', 'Nashik', 'Faridabad', 'Meerut', 'Rajkot']
}

CITY_MULTIPLIERS = {
    'Tier 1': 1.3,  # Higher salaries in metro cities
    'Tier 2': 1.1,  # Moderate increase
    'Tier 3': 1.0   # Base salary
}

SKILLS_CATEGORIES = {
    'Technology': [
        'Python', 'Java', 'JavaScript', 'React', 'Node.js', 'AWS', 'Docker', 
        'Kubernetes', 'SQL', 'MongoDB', 'Machine Learning', 'Data Analysis'
    ],
    'Finance': [
        'Excel', 'Financial Modeling', 'SAP', 'QuickBooks', 'Bloomberg Terminal',
        'Risk Management', 'Compliance', 'Auditing'
    ],
    'Marketing': [
        'Google Analytics', 'SEO', 'SEM', 'Social Media Marketing', 'Content Marketing',
        'Adobe Creative Suite', 'HubSpot', 'Salesforce'
    ],
    'General': [
        'Project Management', 'Leadership', 'Communication', 'Problem Solving',
        'Team Management', 'Strategic Planning', 'Data Analysis'
    ]
}

COMPANY_SIZES = ['Startup (<50)', 'Small (50-200)', 'Medium (200-1000)', 'Large (1000-5000)', 'Enterprise (5000+)']
COMPANY_SIZE_WEIGHTS = [0.15, 0.25, 0.30, 0.20, 0.10]

INDUSTRIES = [
    'Technology', 'Finance', 'Healthcare', 'Manufacturing', 'Retail',
    'Consulting', 'Education', 'Real Estate', 'Media', 'Automotive'
]

print("✅ Configuration and parameters defined!")
print(f"📊 Dataset size: {CONFIG['dataset_size']:,} employees")
print(f"🎯 Target currency: {CONFIG['currency']}")
print(f"📍 Cities covered: {sum(len(cities) for cities in INDIAN_CITIES.values())} across India")
print(f"💼 Job titles: {sum(len(titles) for titles in JOB_TITLES.values())} different roles")

In [ ]:
# Data Generation Functions

def generate_basic_info(n_samples: int) -> pd.DataFrame:
    """Generate basic employee information."""
    data = {
        'employee_id': [f'EMP_{str(i).zfill(6)}' for i in range(1, n_samples + 1)],
        'age': np.random.normal(32, 8, n_samples).astype(int),
        'gender': np.random.choice(GENDER_DIST, n_samples, p=GENDER_WEIGHTS),
        'education_level': np.random.choice(EDUCATION_LEVELS, n_samples, p=EDUCATION_WEIGHTS)
    }
    
    # Ensure age is within reasonable bounds
    data['age'] = np.clip(data['age'], 22, 65)
    
    return pd.DataFrame(data)

def generate_professional_info(basic_df: pd.DataFrame) -> pd.DataFrame:
    """Generate professional information based on age and education."""
    df = basic_df.copy()
    
    # Years of experience (correlated with age and education)
    base_experience = df['age'] - 22  # Assuming work starts at 22
    
    # Adjust for education level
    education_adjustments = {
        'High School': -2, 'Diploma': -1, 'Bachelor': 0, 'Master': 1, 'PhD': 3
    }
    
    experience_adjustments = df['education_level'].map(education_adjustments)
    df['years_experience'] = np.maximum(0, base_experience + experience_adjustments + 
                                      np.random.normal(0, 2, len(df)))
    df['years_experience'] = np.round(df['years_experience'], 1)
    
    # Department (affects salary significantly)
    departments = list(JOB_TITLES.keys())
    dept_weights = [0.35, 0.15, 0.12, 0.15, 0.08, 0.15]  # Technology has highest weight
    df['department'] = np.random.choice(departments, len(df), p=dept_weights)
    
    # Job title based on department and experience
    job_titles = []
    seniority_levels = []
    
    for _, row in df.iterrows():
        dept = row['department']
        exp = row['years_experience']
        
        # Determine seniority level
        if exp < 2:
            seniority = 'Junior'
        elif exp < 5:
            seniority = 'Mid'
        elif exp < 10:
            seniority = 'Senior'
        else:
            seniority = 'Lead'
        
        seniority_levels.append(seniority)
        
        # Select job title from department
        available_titles = JOB_TITLES[dept]
        if seniority == 'Junior':
            # Prefer entry-level titles
            title = np.random.choice(available_titles[:max(1, len(available_titles)//2)])
        elif seniority == 'Lead':
            # Prefer senior titles
            title = np.random.choice(available_titles[len(available_titles)//2:])
        else:
            title = np.random.choice(available_titles)
        
        job_titles.append(title)
    
    df['job_title'] = job_titles
    df['seniority_level'] = seniority_levels
    
    return df

def generate_location_info(df: pd.DataFrame) -> pd.DataFrame:
    """Generate location information."""
    df = df.copy()
    
    # City tier distribution (Tech jobs more in Tier 1)
    tier_probs = []
    for _, row in df.iterrows():
        if row['department'] == 'Technology':
            tier_probs.append([0.6, 0.3, 0.1])  # Higher chance in Tier 1
        elif row['department'] in ['Finance', 'Consulting']:
            tier_probs.append([0.5, 0.35, 0.15])
        else:
            tier_probs.append([0.4, 0.4, 0.2])
    
    city_tiers = []
    cities = []
    states = []
    
    for i, (_, row) in enumerate(df.iterrows()):
        tier = np.random.choice(['Tier 1', 'Tier 2', 'Tier 3'], p=tier_probs[i])
        city = np.random.choice(INDIAN_CITIES[tier])
        
        # Map cities to states (simplified)
        city_state_map = {
            'Mumbai': 'Maharashtra', 'Delhi': 'Delhi', 'Bangalore': 'Karnataka',
            'Hyderabad': 'Telangana', 'Chennai': 'Tamil Nadu', 'Pune': 'Maharashtra',
            'Kolkata': 'West Bengal', 'Ahmedabad': 'Gujarat', 'Jaipur': 'Rajasthan',
            'Surat': 'Gujarat', 'Lucknow': 'Uttar Pradesh', 'Kanpur': 'Uttar Pradesh',
            'Nagpur': 'Maharashtra', 'Indore': 'Madhya Pradesh', 'Coimbatore': 'Tamil Nadu'
        }
        
        state = city_state_map.get(city, 'Other')
        
        city_tiers.append(tier)
        cities.append(city)
        states.append(state)
    
    df['city_tier'] = city_tiers
    df['city'] = cities
    df['state'] = states
    
    return df

def generate_skills_and_performance(df: pd.DataFrame) -> pd.DataFrame:
    """Generate skills and performance metrics."""
    df = df.copy()
    
    # Technical skills score (0-100)
    skills_scores = []
    certifications = []
    
    for _, row in df.iterrows():
        dept = row['department']
        exp = row['years_experience']
        education = row['education_level']
        
        # Base skill score influenced by experience and education
        base_score = min(85, 40 + exp * 3 + {'High School': 0, 'Diploma': 5, 
                                           'Bachelor': 10, 'Master': 15, 'PhD': 20}[education])
        
        # Add randomness
        skill_score = np.clip(base_score + np.random.normal(0, 10), 20, 100)
        skills_scores.append(round(skill_score, 1))
        
        # Certifications (higher for experienced and educated)
        cert_count = 0
        if exp > 3 and np.random.random() < 0.6:
            cert_count += 1
        if education in ['Master', 'PhD'] and np.random.random() < 0.4:
            cert_count += 1
        if dept == 'Technology' and np.random.random() < 0.5:
            cert_count += 1
            
        certifications.append(cert_count)
    
    df['technical_skills_score'] = skills_scores
    df['certifications_count'] = certifications
    
    # Performance rating (1-5 scale)
    # Higher performers tend to earn more
    performance_ratings = np.random.choice([2, 3, 4, 5], len(df), p=[0.05, 0.25, 0.50, 0.20])
    df['performance_rating'] = performance_ratings
    
    # Language skills (English proficiency for Indian market)
    english_levels = ['Basic', 'Intermediate', 'Advanced', 'Native']
    english_weights = [0.1, 0.3, 0.5, 0.1]
    df['english_proficiency'] = np.random.choice(english_levels, len(df), p=english_weights)
    
    return df

def generate_company_info(df: pd.DataFrame) -> pd.DataFrame:
    """Generate company information."""
    df = df.copy()
    
    # Company size affects salary
    df['company_size'] = np.random.choice(COMPANY_SIZES, len(df), p=COMPANY_SIZE_WEIGHTS)
    
    # Industry
    df['industry'] = np.random.choice(INDUSTRIES, len(df))
    
    # Company type
    company_types = ['Private', 'Public', 'Startup', 'MNC', 'Government']
    type_weights = [0.4, 0.15, 0.15, 0.25, 0.05]
    df['company_type'] = np.random.choice(company_types, len(df), p=type_weights)
    
    return df

print("✅ Data generation functions created!")
print("🔧 Functions available:")
print("   • generate_basic_info() - Age, gender, education")
print("   • generate_professional_info() - Experience, job title, department")
print("   • generate_location_info() - City, state, tier")
print("   • generate_skills_and_performance() - Skills, certifications, performance")
print("   • generate_company_info() - Company size, industry, type")

In [ ]:
# Salary Calculation with Business Logic

def calculate_salary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate realistic salaries based on multiple factors.
    
    Salary calculation considers:
    - Base salary by department and seniority
    - Experience multiplier
    - Education bonus
    - Location multiplier
    - Performance bonus
    - Skills premium
    - Company size factor
    """
    df = df.copy()
    
    # Base salaries by department and seniority (in INR)
    base_salaries = {
        'Technology': {'Junior': 400000, 'Mid': 800000, 'Senior': 1500000, 'Lead': 2500000},
        'Finance': {'Junior': 350000, 'Mid': 650000, 'Senior': 1200000, 'Lead': 2000000},
        'Marketing': {'Junior': 300000, 'Mid': 550000, 'Senior': 1000000, 'Lead': 1800000},
        'Sales': {'Junior': 320000, 'Mid': 600000, 'Senior': 1100000, 'Lead': 1900000},
        'HR': {'Junior': 280000, 'Mid': 500000, 'Senior': 900000, 'Lead': 1600000},
        'Operations': {'Junior': 300000, 'Mid': 550000, 'Senior': 1000000, 'Lead': 1700000}
    }
    
    # Education multipliers
    education_multipliers = {
        'High School': 0.85, 'Diploma': 0.95, 'Bachelor': 1.0, 'Master': 1.15, 'PhD': 1.3
    }
    
    # Company size multipliers
    company_size_multipliers = {
        'Startup (<50)': 0.9, 'Small (50-200)': 0.95, 'Medium (200-1000)': 1.0,
        'Large (1000-5000)': 1.1, 'Enterprise (5000+)': 1.2
    }
    
    # Company type multipliers
    company_type_multipliers = {
        'Private': 1.0, 'Public': 1.1, 'Startup': 0.9, 'MNC': 1.25, 'Government': 0.8
    }
    
    salaries = []
    
    for _, row in df.iterrows():
        # Get base salary
        base = base_salaries[row['department']][row['seniority_level']]
        
        # Experience factor (additional 3% per year beyond minimum)
        min_exp_for_level = {'Junior': 0, 'Mid': 2, 'Senior': 5, 'Lead': 10}
        extra_exp = max(0, row['years_experience'] - min_exp_for_level[row['seniority_level']])
        exp_multiplier = 1 + (extra_exp * 0.03)
        
        # Education multiplier
        edu_multiplier = education_multipliers[row['education_level']]
        
        # Location multiplier
        location_multiplier = CITY_MULTIPLIERS[row['city_tier']]
        
        # Performance multiplier
        performance_multiplier = {2: 0.9, 3: 1.0, 4: 1.1, 5: 1.25}[row['performance_rating']]
        
        # Skills premium (0-20% based on technical skills score)
        skills_premium = 1 + (row['technical_skills_score'] / 100) * 0.2
        
        # Certification bonus (5% per certification)
        cert_bonus = 1 + (row['certifications_count'] * 0.05)
        
        # Company factors
        size_multiplier = company_size_multipliers[row['company_size']]
        type_multiplier = company_type_multipliers[row['company_type']]
        
        # Calculate final salary
        calculated_salary = (base * 
                           exp_multiplier * 
                           edu_multiplier * 
                           location_multiplier * 
                           performance_multiplier * 
                           skills_premium * 
                           cert_bonus * 
                           size_multiplier * 
                           type_multiplier)
        
        # Add some randomness (±10%)
        random_factor = np.random.uniform(0.9, 1.1)
        final_salary = calculated_salary * random_factor
        
        # Round to nearest thousand
        final_salary = round(final_salary / 1000) * 1000
        
        salaries.append(final_salary)
    
    df['annual_salary'] = salaries
    
    # Calculate bonus (typically 10-30% of salary)
    bonus_percentages = np.random.uniform(0.1, 0.3, len(df))
    df['annual_bonus'] = (df['annual_salary'] * bonus_percentages).round()
    
    # Total compensation
    df['total_compensation'] = df['annual_salary'] + df['annual_bonus']
    
    return df

def add_data_inconsistencies(df: pd.DataFrame, missing_rate: float = 0.05) -> pd.DataFrame:
    """
    Add realistic data inconsistencies to simulate real-world data.
    
    This includes:
    - Missing values
    - Outliers
    - Inconsistent formatting
    - Data entry errors
    """
    df = df.copy()
    n_samples = len(df)
    
    # Columns that can have missing values (excluding ID and salary)
    columns_for_missing = [
        'technical_skills_score', 'certifications_count', 'english_proficiency', 
        'performance_rating', 'annual_bonus'
    ]
    
    # Introduce missing values
    for col in columns_for_missing:
        if col in df.columns:
            n_missing = int(n_samples * missing_rate * np.random.uniform(0.5, 1.5))
            missing_indices = np.random.choice(df.index, n_missing, replace=False)
            df.loc[missing_indices, col] = np.nan
    
    # Add some outliers in salary (very high earners)
    n_outliers = int(n_samples * 0.01)  # 1% outliers
    outlier_indices = np.random.choice(df.index, n_outliers, replace=False)
    df.loc[outlier_indices, 'annual_salary'] *= np.random.uniform(2, 4, n_outliers)
    df.loc[outlier_indices, 'total_compensation'] = (df.loc[outlier_indices, 'annual_salary'] + 
                                                   df.loc[outlier_indices, 'annual_bonus'].fillna(0))
    
    # Add inconsistent city name formatting (some cities with different cases)
    city_variations = {
        'Mumbai': ['mumbai', 'MUMBAI', 'Mumbai '],
        'Delhi': ['delhi', 'DELHI', 'New Delhi'],
        'Bangalore': ['bangalore', 'BANGALORE', 'Bengaluru']
    }
    
    for city, variations in city_variations.items():
        city_mask = df['city'] == city
        if city_mask.sum() > 0:
            # Change 10% of occurrences to variations
            change_indices = df[city_mask].sample(frac=0.1).index
            for idx in change_indices:
                df.loc[idx, 'city'] = np.random.choice(variations)
    
    return df

print("✅ Salary calculation and data inconsistency functions created!")
print("💰 Salary calculation factors:")
print("   • Base salary by department & seniority")
print("   • Experience multiplier (3% per extra year)")
print("   • Education bonus (up to 30% for PhD)")
print("   • Location premium (up to 30% for Tier 1 cities)")
print("   • Performance bonus (up to 25% for top performers)")
print("   • Skills premium (up to 20% for high technical skills)")
print("   • Company size & type factors")
print("🔧 Data inconsistencies:")
print("   • Missing values (5% rate)")
print("   • Salary outliers (1% high earners)")
print("   • Inconsistent city formatting")

In [ ]:
# Generate Complete Dataset

print("🚀 Starting dataset generation...")
print(f"📊 Generating {CONFIG['dataset_size']:,} employee records...")

# Step 1: Generate basic information
print("\n1️⃣ Generating basic employee information...")
df = generate_basic_info(CONFIG['dataset_size'])
print(f"   ✅ Created {len(df):,} employee records with basic info")

# Step 2: Add professional information
print("2️⃣ Adding professional information...")
df = generate_professional_info(df)
print(f"   ✅ Added experience, job titles, and seniority levels")

# Step 3: Add location information
print("3️⃣ Adding location information...")
df = generate_location_info(df)
print(f"   ✅ Added city, state, and tier information")

# Step 4: Add skills and performance metrics
print("4️⃣ Adding skills and performance metrics...")
df = generate_skills_and_performance(df)
print(f"   ✅ Added technical skills, certifications, and performance ratings")

# Step 5: Add company information
print("5️⃣ Adding company information...")
df = generate_company_info(df)
print(f"   ✅ Added company size, industry, and type")

# Step 6: Calculate salaries
print("6️⃣ Calculating salaries based on all factors...")
df = calculate_salary(df)
print(f"   ✅ Calculated realistic salaries in {CONFIG['currency']}")

# Step 7: Add data inconsistencies
print("7️⃣ Adding realistic data inconsistencies...")
df = add_data_inconsistencies(df, CONFIG['missing_rate'])
print(f"   ✅ Added missing values and inconsistencies")

print(f"\n🎉 Dataset generation completed!")
print(f"📊 Final dataset shape: {df.shape}")
print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display basic statistics
print(f"\n📈 Quick Statistics:")
print(f"   • Average salary: ₹{df['annual_salary'].mean():,.0f}")
print(f"   • Median salary: ₹{df['annual_salary'].median():,.0f}")
print(f"   • Salary range: ₹{df['annual_salary'].min():,.0f} - ₹{df['annual_salary'].max():,.0f}")
print(f"   • Missing values: {df.isnull().sum().sum():,} total")

In [ ]:
# Dataset Overview and Sample

print("📋 Dataset Overview")
print("=" * 50)

# Display first few rows
print("\n🔍 Sample Data (First 5 rows):")
display(df.head())

print(f"\n📊 Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.shape[1]}")
print(f"Rows: {df.shape[0]:,}")

print(f"\n📈 Column Information:")
print(df.info())

print(f"\n🔢 Numerical Summary:")
display(df.describe())

print(f"\n📝 Categorical Summary:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols[:5]:  # Show first 5 categorical columns
    print(f"\n{col}:")
    print(df[col].value_counts().head())

print(f"\n❌ Missing Values Summary:")
missing_summary = df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
if len(missing_summary) > 0:
    print(missing_summary)
else:
    print("No missing values found!")

In [ ]:
# Dataset Visualizations

print("📊 Creating Dataset Visualizations...")

# Set up the plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create figure with subplots
fig = plt.figure(figsize=(20, 15))

# 1. Salary Distribution
plt.subplot(3, 3, 1)
plt.hist(df['annual_salary']/100000, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Salary Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Annual Salary (Lakhs INR)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 2. Salary by Department
plt.subplot(3, 3, 2)
dept_salary = df.groupby('department')['annual_salary'].median().sort_values(ascending=False)
bars = plt.bar(range(len(dept_salary)), dept_salary/100000, color='lightcoral')
plt.title('Median Salary by Department', fontsize=14, fontweight='bold')
plt.xlabel('Department')
plt.ylabel('Median Salary (Lakhs INR)')
plt.xticks(range(len(dept_salary)), dept_salary.index, rotation=45)
plt.grid(True, alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'₹{height:.1f}L', ha='center', va='bottom', fontsize=10)

# 3. Experience vs Salary
plt.subplot(3, 3, 3)
plt.scatter(df['years_experience'], df['annual_salary']/100000, alpha=0.6, color='green')
plt.title('Experience vs Salary', fontsize=14, fontweight='bold')
plt.xlabel('Years of Experience')
plt.ylabel('Annual Salary (Lakhs INR)')
plt.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(df['years_experience'], df['annual_salary']/100000, 1)
p = np.poly1d(z)
plt.plot(df['years_experience'], p(df['years_experience']), "r--", alpha=0.8, linewidth=2)

# 4. Education Level Distribution
plt.subplot(3, 3, 4)
education_counts = df['education_level'].value_counts()
plt.pie(education_counts.values, labels=education_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Education Level Distribution', fontsize=14, fontweight='bold')

# 5. City Tier Distribution
plt.subplot(3, 3, 5)
tier_counts = df['city_tier'].value_counts()
colors = ['gold', 'lightblue', 'lightgreen']
bars = plt.bar(tier_counts.index, tier_counts.values, color=colors)
plt.title('Employee Distribution by City Tier', fontsize=14, fontweight='bold')
plt.xlabel('City Tier')
plt.ylabel('Number of Employees')
plt.grid(True, alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 50,
             f'{int(height):,}', ha='center', va='bottom', fontsize=11)

# 6. Salary by Company Size
plt.subplot(3, 3, 6)
size_salary = df.groupby('company_size')['annual_salary'].median().sort_values()
plt.barh(range(len(size_salary)), size_salary/100000, color='orange', alpha=0.7)
plt.title('Median Salary by Company Size', fontsize=14, fontweight='bold')
plt.xlabel('Median Salary (Lakhs INR)')
plt.ylabel('Company Size')
plt.yticks(range(len(size_salary)), size_salary.index)
plt.grid(True, alpha=0.3)

# 7. Performance Rating Distribution
plt.subplot(3, 3, 7)
perf_counts = df['performance_rating'].value_counts().sort_index()
plt.bar(perf_counts.index, perf_counts.values, color='purple', alpha=0.7)
plt.title('Performance Rating Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Performance Rating')
plt.ylabel('Number of Employees')
plt.grid(True, alpha=0.3)

# 8. Age Distribution
plt.subplot(3, 3, 8)
plt.hist(df['age'], bins=30, alpha=0.7, color='pink', edgecolor='black')
plt.title('Age Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 9. Technical Skills Score Distribution
plt.subplot(3, 3, 9)
plt.hist(df['technical_skills_score'].dropna(), bins=30, alpha=0.7, color='cyan', edgecolor='black')
plt.title('Technical Skills Score Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Technical Skills Score')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Dataset visualizations created!")

In [ ]:
# Interactive Visualizations with Plotly

print("🎨 Creating Interactive Visualizations...")

# 1. Interactive Salary Distribution by Department
fig1 = px.box(df, x='department', y='annual_salary', 
              title='Salary Distribution by Department',
              labels={'annual_salary': 'Annual Salary (INR)', 'department': 'Department'},
              color='department')
fig1.update_layout(height=500, xaxis_tickangle=-45)
fig1.show()

# 2. 3D Scatter Plot: Experience vs Skills vs Salary
fig2 = px.scatter_3d(df.dropna(), x='years_experience', y='technical_skills_score', 
                     z='annual_salary', color='department',
                     title='3D Relationship: Experience vs Skills vs Salary',
                     labels={
                         'years_experience': 'Years of Experience',
                         'technical_skills_score': 'Technical Skills Score',
                         'annual_salary': 'Annual Salary (INR)'
                     })
fig2.update_layout(height=600)
fig2.show()

# 3. Geographic Distribution with Salary Information
city_summary = df.groupby(['city', 'city_tier']).agg({
    'annual_salary': ['mean', 'median', 'count']
}).round(0)
city_summary.columns = ['avg_salary', 'median_salary', 'employee_count']
city_summary = city_summary.reset_index()

fig3 = px.scatter(city_summary, x='employee_count', y='median_salary',
                  size='avg_salary', color='city_tier',
                  hover_name='city',
                  title='City Analysis: Employee Count vs Median Salary',
                  labels={
                      'employee_count': 'Number of Employees',
                      'median_salary': 'Median Salary (INR)',
                      'city_tier': 'City Tier'
                  })
fig3.update_layout(height=500)
fig3.show()

# 4. Correlation Heatmap
numerical_cols = ['age', 'years_experience', 'technical_skills_score', 
                  'performance_rating', 'certifications_count', 'annual_salary']
correlation_matrix = df[numerical_cols].corr()

fig4 = px.imshow(correlation_matrix, 
                 text_auto=True, aspect="auto",
                 title='Correlation Matrix of Numerical Features',
                 color_continuous_scale='RdBu_r')
fig4.update_layout(height=500)
fig4.show()

# 5. Salary Trends by Multiple Factors
fig5 = px.sunburst(df, path=['department', 'seniority_level', 'education_level'], 
                   values='annual_salary',
                   title='Salary Distribution Hierarchy: Department → Seniority → Education')
fig5.update_layout(height=600)
fig5.show()

print("✅ Interactive visualizations created!")

In [ ]:
# Save Dataset and Generate Report

# Create data directory if it doesn't exist
import os
os.makedirs(CONFIG['save_path'], exist_ok=True)

# Save the main dataset
main_file_path = os.path.join(CONFIG['save_path'], 'employee_salary_dataset.csv')
df.to_csv(main_file_path, index=False)

print(f"💾 Dataset saved to: {main_file_path}")
print(f"📊 File size: {os.path.getsize(main_file_path) / 1024**2:.2f} MB")

# Create a clean version without inconsistencies for comparison
df_clean = df.copy()

# Fix city name inconsistencies
city_corrections = {
    'mumbai': 'Mumbai', 'MUMBAI': 'Mumbai', 'Mumbai ': 'Mumbai',
    'delhi': 'Delhi', 'DELHI': 'Delhi', 'New Delhi': 'Delhi',
    'bangalore': 'Bangalore', 'BANGALORE': 'Bangalore', 'Bengaluru': 'Bangalore'
}

for old_name, new_name in city_corrections.items():
    df_clean['city'] = df_clean['city'].replace(old_name, new_name)

# Save clean version
clean_file_path = os.path.join(CONFIG['save_path'], 'employee_salary_dataset_clean.csv')
df_clean.to_csv(clean_file_path, index=False)

print(f"🧹 Clean dataset saved to: {clean_file_path}")

# Generate Dataset Report
report = {
    'generation_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_info': {
        'total_records': len(df),
        'total_features': len(df.columns),
        'file_size_mb': round(os.path.getsize(main_file_path) / 1024**2, 2),
        'currency': CONFIG['currency']
    },
    'feature_summary': {
        'numerical_features': len(df.select_dtypes(include=[np.number]).columns),
        'categorical_features': len(df.select_dtypes(include=['object']).columns),
        'missing_values_total': int(df.isnull().sum().sum()),
        'missing_percentage': round(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100, 2)
    },
    'salary_statistics': {
        'mean_salary': float(df['annual_salary'].mean()),
        'median_salary': float(df['annual_salary'].median()),
        'min_salary': float(df['annual_salary'].min()),
        'max_salary': float(df['annual_salary'].max()),
        'std_salary': float(df['annual_salary'].std())
    },
    'demographics': {
        'age_range': f"{df['age'].min()}-{df['age'].max()}",
        'experience_range': f"{df['years_experience'].min():.1f}-{df['years_experience'].max():.1f}",
        'gender_distribution': df['gender'].value_counts().to_dict(),
        'education_distribution': df['education_level'].value_counts().to_dict()
    },
    'professional_breakdown': {
        'departments': df['department'].value_counts().to_dict(),
        'seniority_levels': df['seniority_level'].value_counts().to_dict(),
        'top_job_titles': df['job_title'].value_counts().head(10).to_dict()
    },
    'geographic_distribution': {
        'city_tiers': df['city_tier'].value_counts().to_dict(),
        'top_cities': df['city'].value_counts().head(10).to_dict(),
        'states': df['state'].value_counts().to_dict()
    },
    'data_quality': {
        'duplicate_records': int(df.duplicated().sum()),
        'outliers_detected': int((df['annual_salary'] > df['annual_salary'].quantile(0.99)).sum()),
        'inconsistent_cities': len(set(df['city']) - set(sum(INDIAN_CITIES.values(), []))),
        'validation_status': 'Generated with realistic business logic'
    }
}\n\n# Save report as JSON\nimport json\nreport_file_path = os.path.join(CONFIG['save_path'], 'dataset_generation_report.json')\nwith open(report_file_path, 'w') as f:\n    json.dump(report, f, indent=4)\n\nprint(f\"📋 Generation report saved to: {report_file_path}\")\n\n# Display summary\nprint(\"\\n\" + \"=\"*60)\nprint(\"📊 DATASET GENERATION SUMMARY\")\nprint(\"=\"*60)\nprint(f\"✅ Successfully generated {report['dataset_info']['total_records']:,} employee records\")\nprint(f\"📁 Main dataset: {main_file_path}\")\nprint(f\"🧹 Clean dataset: {clean_file_path}\")\nprint(f\"📋 Report: {report_file_path}\")\nprint(f\"\\n💰 Salary Statistics (INR):\")\nprint(f\"   • Average: ₹{report['salary_statistics']['mean_salary']:,.0f}\")\nprint(f\"   • Median: ₹{report['salary_statistics']['median_salary']:,.0f}\")\nprint(f\"   • Range: ₹{report['salary_statistics']['min_salary']:,.0f} - ₹{report['salary_statistics']['max_salary']:,.0f}\")\nprint(f\"\\n🏢 Department Distribution:\")\nfor dept, count in report['professional_breakdown']['departments'].items():\n    print(f\"   • {dept}: {count:,} ({count/report['dataset_info']['total_records']*100:.1f}%)\")\nprint(f\"\\n🌍 Geographic Distribution:\")\nfor tier, count in report['geographic_distribution']['city_tiers'].items():\n    print(f\"   • {tier}: {count:,} ({count/report['dataset_info']['total_records']*100:.1f}%)\")\nprint(f\"\\n⚠️  Data Quality:\")\nprint(f\"   • Missing values: {report['feature_summary']['missing_values_total']:,} ({report['feature_summary']['missing_percentage']:.1f}%)\")\nprint(f\"   • Outliers: {report['data_quality']['outliers_detected']:,}\")\nprint(f\"   • Duplicates: {report['data_quality']['duplicate_records']:,}\")\nprint(\"\\n🎉 Dataset generation completed successfully!\")\nprint(\"🚀 Ready for data cleaning, EDA, and machine learning modeling!\")